In [12]:
using Pkg
Pkg.activate("..")

  Activating project at `e:\QuantumHarmonicBalance`


In [13]:
using QuantumHarmonicBalance; QHB = QuantumHarmonicBalance
using QuantumCumulants; QC = QuantumCumulants
import Plots
using SymbolicUtils, OrdinaryDiffEq

In [14]:
# Hilbert space
h = FockSpace(:cavity)

@cnumbers Δ F κ ω ω₀ α
@qnumbers a::Destroy(h)
@syms t::Real

(t,)

In [15]:
expr = ω₀*a'*a + α*(a'+ a)^4/4 + F*(a'+ a)*cos(ω*t) |> simplify
rot = rotate(expr, a, ω, t)

(0.5F*(a′)+0.5F*(a)+0.75α+1.5α*(a′*a′*a*a)+3.0α*(a′*a)+ω₀*(a′*a)+0.5F*exp((0 + 2im)*t*ω)*(a)+0.5F*exp((0 - 2im)*t*ω)*(a′)+1.5exp((0 + 2im)*t*ω)*α*(a*a)+exp((0 + 2im)*t*ω)*α*(a′*a*a*a)+exp((0 - 2im)*t*ω)*α*(a′*a′*a′*a)+1.5exp((0 - 2im)*t*ω)*α*(a′*a′)+0.25exp((0 - 4im)*t*ω)*α*(a′*a′*a′*a′)+0.25exp((0 + 4im)*t*ω)*α*(a*a*a*a)+-ω*(a′*a))

In [16]:
rot_av = QHB.average(rot) |> QHB.remove_constants
components = get_fourier_components(rot_av, ω, t)

Dict{Int64, Union{Number, QSym, QTerm, SymbolicUtils.BasicSymbolic}} with 5 entries:
  0  => (0.5F*(a′)+0.5F*(a)+1.5α*(a′*a′*a*a)+3.0α*(a′*a)+-ω*(a′*a)+ω₀*(a′*a))
  4  => 0.25α*(a*a*a*a)
  2  => (0.5F*(a)+1.5α*(a*a)+α*(a′*a*a*a))
  -2 => (0.5F*(a′)+α*(a′*a′*a′*a)+1.5α*(a′*a′))
  -4 => 0.25α*(a′*a′*a′*a′)

In [18]:
Hrwa  =rotating_wave_approximation(expr, a, ω, t)

(0.5F*(a′)+0.5F*(a)+1.5α*(a′*a′*a*a)+3.0α*(a′*a)+-ω*(a′*a)+ω₀*(a′*a))

In [21]:
eqs_RWA = meanfield([a], Hrwa; order=1) |> complete

∂ₜ(⟨a⟩) = (0.0 - 0.5im)*F + (0.0 - 3.0im)*⟨a⟩*α + (0 + 1im)*⟨a⟩*ω + (0 - 1im)*⟨a⟩*ω₀ + (0.0 - 3.0im)*⟨a′⟩*(⟨a⟩^2)*α


In [24]:
eqs_RWA |> typeof |> fieldnames

(:equations, :operator_equations, :states, :operators, :hamiltonian, :jumps, :jumps_dagger, :rates, :iv, :varmap, :order)

In [25]:
eqs_RWA.equations

1-element Vector{Symbolics.Equation}:
 ⟨a⟩ ~ (0.0 - 0.5im)*F + (0.0 - 3.0im)*⟨a⟩*α + (0 + 1im)*⟨a⟩*ω + (0 - 1im)*⟨a⟩*ω₀ + (0.0 - 3.0im)*⟨a′⟩*(⟨a⟩^2)*α